In [ ]:
import nltk
from nltk.tokenize import RegexpTokenizer
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer
import re

nltk.download('averaged_perceptron_tagger')

nltk.download('maxent_ne_chunker')
nltk.download('words')

### Preprocessing

#### Bookwise preprocessing

In [ ]:
nltk.__version__

In [ ]:
books = ['Outlander', 'Dragonfly in Amber', 'Voyager', 'Drums of Autumn', 'The Fiery Cross', 'A Breath of Snow and Ashes',
         'An Echo in the Bone', 'Written in My Own Heart’s Blood']
extras = ['Other Books by this Author', 'About the Author']
bookstarts = [50, 17287, 37378, 61857, 89432, 119494, 152540, 177800, 202059]

In [ ]:
f = open("data.txt", "r", encoding="utf8")
booknum = 1


book = ''
i = 0
for line in f:
    if i < bookstarts[booknum] and i > bookstarts[booknum-1]:
        book = book + line
    i = i + 1

In [ ]:
# we want to segment the data into chapters as we want to see the topic per chapter. But how do we do that?
# the author devided the books into parts, this is how we can see when hte first chapter begins, then we search for a line
# that is written in all caps, after this the chapter starts: example: \n\n\n\n\n\n1\n\nA NEW BEGINNING\n\n
#
# book 1 has 41, 49, 63, 71, 111, 124, 103, 145
# found is 41, 49, 63, 71, 111, 124, 0, 0
def split_into_chapters(booknum, text):
    if booknum <7:
        return re.split(r'\n[0-9]+\n\n', book)
    else:
        # needs refinement
        return re.split(r'[A-Z ]+\n\n', book)

In [ ]:
def split_into_paragraphs(text):
    splitted = re.split('(\*\*\*)', text)
    return [x for x in splitted if x != '']

In [ ]:
# print(split_into_paragraphs(split_into_chapters(1, book)[4]))
sentences = split_into_paragraphs(split_into_chapters(1, book)[0])
sentences[0]


#### Linguistic Preprocessing

In [ ]:
# tokenizer
per_paragraph = True

tokenizer = RegexpTokenizer(r"[a-zA-Z’]+")
chapters = split_into_chapters(booknum, book)
tchapters = []
for chapter in chapters:
    tparagraphs = []
    if per_paragraph:
        paragraphs = split_into_paragraphs(chapter)
        for paragraph in paragraphs:
            tparagraphs.append(tokenizer.tokenize(paragraph))
        tchapters.append(tparagraphs)
    else:
        tchapters.append(tokenizer.tokenize(chapter))

In [ ]:
chapternum = 0
lemmatizer=WordNetLemmatizer()
if not per_paragraph:
    for word in tchapters[chapternum]:
        print(lemmatizer.lemmatize(word))
else:
    for paragraph in tchapters[chapternum]:
        for word in paragraph:
            print(lemmatizer.lemmatize(word))

## FLAIR NER Tagging

In [ ]:
!pip install flair

In [ ]:
from flair.data import Sentence, Token
from flair.models import SequenceTagger


In [ ]:
tagger = SequenceTagger.load("flair/ner-english-ontonotes-fast")

locations = []
chapters = re.split(r'\n[0-9]+\n\n', book)
l = 0
for chapter in chapters:
    sentence = Sentence(chapter)
    tagger.predict(sentence)
    for entity in sentence.get_spans('ner'):
        with open('ner1.txt', 'a') as f:
                f.write(entity.text)
                f.write(entity.tag)
                f.write(str(entity.start_position+l))
                f.write('\n')
        if entity.tag=='GPE':
            locations.append(entity.text)
            with open('locs1.txt', 'a') as f:
                f.write(entity.text)
                f.write('\n')
    l += len(chapter)
